# A personal project using plotly to create an interactive visual that changes based on what dropdown is selected. Reads an excel sheet with millions of entries and shows the breakdown of that job's payroll.

In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.io as pio

In [11]:
pay = pd.read_csv('/Users/Carlos/Desktop/Project_Files/Citywide_Payroll_Data__Fiscal_Year_.csv')

In [12]:
#pay = payroll.iloc[:500000, :] use slicing to make the df smaller for quicker testing.
pay['Base Salary'] = pay['Base Salary'].replace(('\,', '\.(.*)'), '', regex = True).astype(np.int64)
pay = pay.where(pay['Pay Basis'] == 'per Annum').dropna(how = 'all', axis = 0)

In [13]:
mean = pay.groupby('Title Description')['Base Salary'].mean()
median = pay.groupby('Title Description')['Base Salary'].median()
minsalary = pay.groupby('Title Description')['Base Salary'].min()
maxsalary = pay.groupby('Title Description')['Base Salary'].max()
count = pay.groupby('Title Description')['Base Salary'].count()
frame = pd.DataFrame({'Mean' : mean, 'Median' : median, 'Minimum Salary' : minsalary, 'Maximum Salary' : maxsalary, 'Count' : count})
frame.reset_index(level= 'Title Description', inplace = True)
pd.DataFrame(frame)

,Title Description,Mean,Median,Minimum Salary,Maximum Salary,Count
0,* ATTENDING DENTIST,144040.000000,144040.0,144040.0,144040.0,1
1,*ADM DIR FLEET MAINT-MGRL ASGMNT,118034.000000,118034.0,118034.0,118034.0,2
2,*ADM DIR FLEET MAINTENANCE - NM,132561.125000,129158.0,129039.0,142358.0,8
3,*ADM SCHOOL SECURITY MANAGER-U,78918.000000,82309.5,65886.0,85725.0,12
4,*ADMIN SCHL SECUR MGR-MGL,149058.000000,143850.0,140341.0,162983.0,3
...,...,...,...,...,...,...
1582,WIPER,84244.340426,85825.0,79275.0,88400.0,94
1583,WORKER'S COMPENSATION BENEFITS EXAMINER,47634.402667,47091.0,34898.0,64584.0,375
1584,X-RAY TECHNICIAN,55277.750000,54624.0,45320.0,64679.0,64
1585,YOUTH COORDINATOR,56687.733333,56650.0,50000.0,61508.0,60


In [14]:
#Similar dataframe using .describe(), won't be used, just showing an alternative.
payer = pay.groupby('Title Description')['Base Salary'].apply(lambda x: pd.Series(x.values)).unstack().T
payer = payer.describe().T
payer.reset_index(level= 'Title Description', inplace = True)

In [18]:
payer

,Title Description,count,mean,std,min,25%,50%,75%,max
0,* ATTENDING DENTIST,1.0,144040.000000,NaN,144040.0,144040.00,144040.0,144040.00,144040.0
1,*ADM DIR FLEET MAINT-MGRL ASGMNT,2.0,118034.000000,0.000000,118034.0,118034.00,118034.0,118034.00,118034.0
2,*ADM DIR FLEET MAINTENANCE - NM,8.0,132561.125000,6062.407924,129039.0,129039.00,129158.0,133344.50,142358.0
3,*ADM SCHOOL SECURITY MANAGER-U,12.0,78918.000000,7642.367708,65886.0,71606.00,82309.5,85725.00,85725.0
4,*ADMIN SCHL SECUR MGR-MGL,3.0,149058.000000,12186.364880,140341.0,142095.50,143850.0,153416.50,162983.0
...,...,...,...,...,...,...,...,...,...
1582,WIPER,94.0,84244.340426,4190.756756,79275.0,79275.00,85825.0,88400.00,88400.0
1583,WORKER'S COMPENSATION BENEFITS EXAMINER,375.0,47634.402667,6287.014986,34898.0,43046.50,47091.0,51991.00,64584.0
1584,X-RAY TECHNICIAN,64.0,55277.750000,4656.655691,45320.0,52080.25,54624.0,57864.75,64679.0
1585,YOUTH COORDINATOR,60.0,56687.733333,2056.343199,50000.0,55034.00,56650.0,57925.00,61508.0


In [15]:
paymath = pay.merge(frame, how = 'left')

# The code below is used to create the interactive visual, but the transition from Jupyter Notebook to GitHub removes the interactive elements. Please refer to the README.md file for more information.

In [16]:
fig = go.Figure()

In [17]:
use_date = widgets.Dropdown(
options = list(paymath['Fiscal Year'].unique()),
description = 'Year : ',
value = 2016.0)

job_list = list(paymath['Title Description'].unique())
status_list = list(paymath['Leave Status as of June 30'].unique())

container = widgets.HBox(childern = [use_date])

status = widgets.Dropdown(
    options = status_list,
    value = 'CEASED',
    description = 'Status : '
)


title = widgets.Dropdown(
    options = job_list,
    value = '* ATTENDING DENTIST',
    description = 'Job Title:   ',
)

filter_list = [i and j and k for i, j, k in zip(paymath['Fiscal Year'] == 2016.0,
                                                paymath['Leave Status as of June 30'] == 'CEASED',
                                                paymath['Title Description'] == '*ATTENDING DENTIST')]

trace1 = go.Scatter(x = paymath[filter_list]['Work Location Borough'], y = paymath[filter_list]['Base Salary'], 
                    mode = 'markers', marker_size = 10, name = 'Distribution')
trace2 = go.Bar(x = paymath[filter_list]['Mean'], y = paymath[filter_list]['Base Salary'], name = 'Mean')
trace3 = go.Bar(x = paymath[filter_list]['Median'], y = paymath[filter_list]['Base Salary'], name = 'Median')
trace4 = go.Bar(x = paymath[filter_list]['Minimum Salary'], y = paymath[filter_list]['Base Salary'], name = 'Minimum')
trace5 = go.Bar(x = paymath[filter_list]['Maximum Salary'], y = paymath[filter_list]['Base Salary'], name = 'Maximum')



g = go.FigureWidget(data = [trace1, trace2, trace3, trace4, trace5], layout = go.Layout(width = 550, height = 550, font_size = 12, 
                                                        title = dict(text = 'NYC Payroll by Borough')))

def validate():
    if title.value in paymath['Title Description'].unique():
        return True
    else:
        return False

def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in zip(paymath['Fiscal Year'] == use_date.value, 
                                                            paymath['Leave Status as of June 30'] == status.value,
                                                            paymath['Title Description'] == title.value)]
            temp_df = paymath[filter_list]
        else:
            filter_list = [i and j for i,j in zip(paymath['Leave Status as of June 30'] == status.value,
                                                  paymath['Title Description'] == title.value)]
            temp_df = paymath[filter_list]
            
        with g.batch_update():
            g.data[0].x = temp_df['Work Location Borough']
            g.data[0].y = temp_df['Base Salary']
            g.data[1].x = ['Citywide Mean']
            g.data[1].y = temp_df['Mean']
            g.data[2].x = ['Citywide Median']
            g.data[2].y = temp_df['Median']
            g.data[3].x = ['Citywide Minimum']
            g.data[3].y = temp_df['Minimum Salary']
            g.data[4].x = ['Citywide Maximum']
            g.data[4].y = temp_df['Maximum Salary']
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Location & Basic Math'
            g.layout.yaxis.title = 'Salary'
            
title.observe(response, names = "value")
status.observe(response, names = "value")
use_date.observe(response, names = "value")

container2 = widgets.HBox([title, status])
widgets.VBox([container, container2, g], renderer = 'svg')